In [97]:
from twython import Twython, TwythonError
import pandas as pd
import csv
import math

In [3]:
API_key = "M3ptGiuDD8KoIldEFCAkImEGi"

API_secret_key = "qvTDEypEKXOpzuFCpY9DrPu4OTItPgzOA6OXAbU0ToYz4P2X0E"

In [4]:
twitter = Twython(API_key, API_secret_key)

In [113]:
def hydrate_database(source, destination, twitter, n_tweets=100):
    """ Takes a csv file of tweet_ids and assiociated labels, recover the tweet content using a Twython object
        and writes it in the destination file in the csv format with columns : 
            tweet_id, label, tweet_content
        If the tweet is retweeted, we recover the original tweet, since the retweeted version might have been truncated.
    """
    
    df = pd.read_csv(source)
    labels = dict(zip(df["tweet_id"].values, df["label"].values))
    destination = open(destination, "w")
    destination.write("tweet_id\tlabel\ttweet_content\n")
    entries = 0
    retrieved = 0
    for i in range( math.ceil( len(df)/n_tweets )):
        imin = i * n_tweets
        imax = min( len(df), (i+1) * n_tweets )
        entries += imax-imin
        print("Retrieved:\t{:5d} /{:5d}".format(retrieved, entries),end='\r', flush=True)
        try :
            tweets = twitter.lookup_status(id=list(df["tweet_id"].values[imin:imax]))
        except TwythonError as e:
            print("Error", e.error_code)
            continue
        retrieved += len(tweets)
        for tweet in tweets:
            tweet_id = tweet["id"]
            if "retweeted_status" in tweet.keys():
                tweet = tweet["retweeted_status"]
            destination.write("{}\t{}\t{}\n".format(tweet["id"], labels[tweet_id], '\"'+repr(tweet["text"])[1:-1].replace('\t',' ')+'\"' ))

    destination.close()

In [114]:
hydrate_database("data/hatespeech_labels.csv", "data/test.csv", twitter)